In [3]:
import os
import pandas as pd
import polars as pl
import numpy as np

In [4]:
scc_threshold_rank = 100
wash_window_sizes_seconds = [3600, 86400, 604800]
wash_trade_detection_ether = False
wash_trade_detection_margin = 0.01

In [5]:
data = "data/IDEXTrades-preprocessed.csv" # "data/IDEXTrades-preprocessed.csv" #
output_folder = "output_test"

global_ether_id = "0x0000000000000000000000000000000000000000"

global_trader_hashes = pd.DataFrame(columns=['trader_address', 'trader_id'])
global_scc_traders_map = {}

In [6]:
os.makedirs(output_folder, exist_ok=True)

## Load and Process Data

In [4]:
def load_trades(file_csv="data/EtherDeltaTrades-preprocessed.csv"):
    trades = pd.read_csv(file_csv)
    print(f"Info: read file {file_csv} as DataFrame with {len(trades)} rows.")
    print(f"Columns are: {', '.join(trades.columns)}")
    return trades

trades = load_trades(data)
trades

/tmp/ipykernel_324543/2556080108.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  trades = pd.read_csv(file_csv)


Info: read file data/IDEXTrades-preprocessed.csv as DataFrame with 5344149 rows.
Columns are: blockNumber, timestamp, transactionHash, status, maker, taker, tokenBuy, tokenSell, amountBuyReal, amountBoughtReal, amountSellReal, amountSoldReal, price, feeMake, feeTake, gas, gasPrice, nonce, tradeNonce, expires


,blockNumber,timestamp,transactionHash,status,maker,taker,tokenBuy,tokenSell,amountBuyReal,amountBoughtReal,amountSellReal,amountSoldReal,price,feeMake,feeTake,gas,gasPrice,nonce,tradeNonce,expires
0,4317409,1506553069,0x87bbc5fe23574dc757ab818a9011cf6c18bf5066f577...,NaN,0x034767f3c519f361c5ecf46ebfc08981c629d381,0x034767f3c519f361c5ecf46ebfc08981c629d381,0x0000000000000000000000000000000000000000,0x744d70fdbe2ba4cf95131626614a1763df805b9e,0.100000,0.100000,1.000000,1.000000,10.000000,0.001,0.010500,290000,5000000000,2496,2510,190000
1,4317416,1506553483,0xbbe1a352d0edd1e9c9352c08887f5858571678ee978d...,NaN,0x74719d7f5c54f747c6f8c18ce948d641c90a037b,0x034767f3c519f361c5ecf46ebfc08981c629d381,0x0000000000000000000000000000000000000000,0x744d70fdbe2ba4cf95131626614a1763df805b9e,0.000495,0.000495,5.000000,5.000000,10093.873019,0.001,0.100000,290000,5000000000,7,2520,190000
2,4317434,1506554023,0xcf568721a37b63e5a09c378b88ae563e099144a16f46...,NaN,0x034767f3c519f361c5ecf46ebfc08981c629d381,0x034767f3c519f361c5ecf46ebfc08981c629d381,0x0000000000000000000000000000000000000000,0x744d70fdbe2ba4cf95131626614a1763df805b9e,0.010000,0.010000,1.000000,1.000000,100.000000,0.001,0.087000,290000,5000000000,2529,2533,190000
3,4317434,1506554023,0xa77c2bc65be11c2df001355b2cd2f28cbb9c5c790f10...,NaN,0x034767f3c519f361c5ecf46ebfc08981c629d381,0x034767f3c519f361c5ecf46ebfc08981c629d381,0x0000000000000000000000000000000000000000,0x744d70fdbe2ba4cf95131626614a1763df805b9e,0.010000,0.010000,1.000000,1.000000,100.000000,0.001,0.087000,290000,5000000000,2523,2531,190000
4,4317434,1506554023,0x241490f67b1c76fe86a61cc0725c012407ab51f12a38...,NaN,0x034767f3c519f361c5ecf46ebfc08981c629d381,0x034767f3c519f361c5ecf46ebfc08981c629d381,0x0000000000000000000000000000000000000000,0x744d70fdbe2ba4cf95131626614a1763df805b9e,0.010000,0.010000,1.000000,1.000000,100.000000,0.001,0.087000,290000,5000000000,2526,2532,190000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5344144,9999990,1588598385,0xe25d774ce7eedbdbdcd6d21dc478c1ea1f35c2a1ac54...,1.0,0x79147f4a41d558d2340034f7ef41525edd2966d0,0x4aa08b97955ddd3829ad17eb930c1c8903cf20ea,0x0000000000000000000000000000000000000000,0xd947b0ceab2a8885866b9a04a06ae99de852a3d4,0.254160,0.254160,1632.056635,1632.056635,6421.371829,0.001,0.010026,400000,11000000000,1588598108158596,27,100000
5344145,9999997,1588598492,0xf049d457a35e6ab08430662240f133e05d4d74e920f2...,1.0,0x7d825166cd469d49f480ff208183fa7546d2825b,0x12d6956c30b8309c6396fb47ac500dc8e1f793fc,0x0000000000000000000000000000000000000000,0x4a220e6096b25eadb88358cb44068a3248254675,4.105321,0.344795,175.026394,14.700000,42.634033,0.001,0.007917,400000,11000000000,1588609310996,94,10000
5344146,9999997,1588598492,0xeadc59e8407b1566ae755a6a980cc971f12fed7a9c6d...,1.0,0xaec346cf77ccfad030f4e83db29e8624c65ed295,0x12d6956c30b8309c6396fb47ac500dc8e1f793fc,0x0000000000000000000000000000000000000000,0x4a220e6096b25eadb88358cb44068a3248254675,0.211108,0.211108,9.000183,9.000183,42.633137,0.001,0.011663,400000,11000000000,51334745949,93,10000
5344147,9999998,1588598524,0xe63bd29f97a3fd9035a9581ed4ca6fecb310cceee4d7...,1.0,0xe706b6071a5459fa34f7c85d938edd1912a4cf56,0x9fee99fa1eb0c0e7ddbba00ce764ae1fe9bd2684,0x0000000000000000000000000000000000000000,0xe5caef4af8780e59df925470b050fb23c43ca68c,0.887616,0.887616,14482.000000,14482.000000,16315.609143,0.000,0.004298,400000,11000000000,351037383,119272554,100000


In [5]:
def get_successful_and_complete_trades(trades, status_column=None, status_success=None):
    n = len(trades)
    if status_column and status_success:
        trades = trades[trades[status_column] == status_success]
    trades = trades.dropna()
    print(f"Info: dropped {n - len(trades)} rows with missing/unsuccessful status. {len(trades)} rows remaining.")
    return trades


if 'EtherDelta' in data:
    _trades = get_successful_and_complete_trades(trades)
else:
    _trades = get_successful_and_complete_trades(trades, 'status', 1)
_trades

Info: dropped 2936 rows with missing/unsuccessful status. 5341213 rows remaining.


,blockNumber,timestamp,transactionHash,status,maker,taker,tokenBuy,tokenSell,amountBuyReal,amountBoughtReal,amountSellReal,amountSoldReal,price,feeMake,feeTake,gas,gasPrice,nonce,tradeNonce,expires
575,4371302,1508159416,0xdcddd3aee6ccc5df4a59d46cb3742509dd5ab2cc22ed...,1.0,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,6.500000,1.500000,5000.000000,1153.846154,769.230769,0.001,0.003133,290000,10000000000,208,232,190000
576,4371451,1508162021,0xba3fc442e5df6b53d011983511644b2582f99b64dd19...,1.0,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,7.500000,7.500000,5000.000000,5000.000000,666.666667,0.001,0.002181,290000,8000000000,212,233,190000
577,4371467,1508162270,0x54a5606d69477b310e74347696b854bd64ae44124d4b...,1.0,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,7.750000,7.750000,5000.000000,5000.000000,645.161290,0.001,0.002175,290000,8000000000,210,234,190000
578,4371574,1508164311,0xd18c4cc92549568e00f0f22a829c98bb16aad45f336d...,1.0,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x55475d4434495a93521afb24e67483f0d05701ba,0x514910771af9ca656af840dff83e8264ecf986ca,0x0000000000000000000000000000000000000000,5000.000000,350.000000,6.000000,0.420000,0.001200,0.001,0.005238,290000,8000000000,242,22,190000
579,4372621,1508180974,0xfe55e8da4fd807d2a25518a2ff9bab09fa34e763ea0d...,1.0,0xe7545f76ecbc8435d348c79db53ea6e6693578ff,0xcd68b4f73fd20e6d50dc1dbec229addcba50554a,0x0000000000000000000000000000000000000000,0xb97048628db6b661d4c2aa833e95dbe1a905b280,0.370104,0.060278,62.320000,10.150000,168.385206,0.001,0.013281,290000,4000000000,1692,1,190000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5344144,9999990,1588598385,0xe25d774ce7eedbdbdcd6d21dc478c1ea1f35c2a1ac54...,1.0,0x79147f4a41d558d2340034f7ef41525edd2966d0,0x4aa08b97955ddd3829ad17eb930c1c8903cf20ea,0x0000000000000000000000000000000000000000,0xd947b0ceab2a8885866b9a04a06ae99de852a3d4,0.254160,0.254160,1632.056635,1632.056635,6421.371829,0.001,0.010026,400000,11000000000,1588598108158596,27,100000
5344145,9999997,1588598492,0xf049d457a35e6ab08430662240f133e05d4d74e920f2...,1.0,0x7d825166cd469d49f480ff208183fa7546d2825b,0x12d6956c30b8309c6396fb47ac500dc8e1f793fc,0x0000000000000000000000000000000000000000,0x4a220e6096b25eadb88358cb44068a3248254675,4.105321,0.344795,175.026394,14.700000,42.634033,0.001,0.007917,400000,11000000000,1588609310996,94,10000
5344146,9999997,1588598492,0xeadc59e8407b1566ae755a6a980cc971f12fed7a9c6d...,1.0,0xaec346cf77ccfad030f4e83db29e8624c65ed295,0x12d6956c30b8309c6396fb47ac500dc8e1f793fc,0x0000000000000000000000000000000000000000,0x4a220e6096b25eadb88358cb44068a3248254675,0.211108,0.211108,9.000183,9.000183,42.633137,0.001,0.011663,400000,11000000000,51334745949,93,10000
5344147,9999998,1588598524,0xe63bd29f97a3fd9035a9581ed4ca6fecb310cceee4d7...,1.0,0xe706b6071a5459fa34f7c85d938edd1912a4cf56,0x9fee99fa1eb0c0e7ddbba00ce764ae1fe9bd2684,0x0000000000000000000000000000000000000000,0xe5caef4af8780e59df925470b050fb23c43ca68c,0.887616,0.887616,14482.000000,14482.000000,16315.609143,0.000,0.004298,400000,11000000000,351037383,119272554,100000


In [6]:
def get_ether_token_trades(trades, token_column1, token_column2):
    n = len(trades)
    trades = trades[(trades[token_column1] == global_ether_id) | (trades[token_column2] == global_ether_id)]
    trades = trades[trades[token_column1] != trades[token_column2]]
    print(f"Info: dropped {n - len(trades)} rows between two tokens or same currency trades. {len(trades)} rows remaining.")
    return trades

__trades = get_ether_token_trades(_trades, 'tokenBuy', 'tokenSell')
__trades

Info: dropped 676 rows between two tokens or same currency trades. 5340537 rows remaining.


,blockNumber,timestamp,transactionHash,status,maker,taker,tokenBuy,tokenSell,amountBuyReal,amountBoughtReal,amountSellReal,amountSoldReal,price,feeMake,feeTake,gas,gasPrice,nonce,tradeNonce,expires
575,4371302,1508159416,0xdcddd3aee6ccc5df4a59d46cb3742509dd5ab2cc22ed...,1.0,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,6.500000,1.500000,5000.000000,1153.846154,769.230769,0.001,0.003133,290000,10000000000,208,232,190000
576,4371451,1508162021,0xba3fc442e5df6b53d011983511644b2582f99b64dd19...,1.0,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,7.500000,7.500000,5000.000000,5000.000000,666.666667,0.001,0.002181,290000,8000000000,212,233,190000
577,4371467,1508162270,0x54a5606d69477b310e74347696b854bd64ae44124d4b...,1.0,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,7.750000,7.750000,5000.000000,5000.000000,645.161290,0.001,0.002175,290000,8000000000,210,234,190000
578,4371574,1508164311,0xd18c4cc92549568e00f0f22a829c98bb16aad45f336d...,1.0,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x55475d4434495a93521afb24e67483f0d05701ba,0x514910771af9ca656af840dff83e8264ecf986ca,0x0000000000000000000000000000000000000000,5000.000000,350.000000,6.000000,0.420000,0.001200,0.001,0.005238,290000,8000000000,242,22,190000
579,4372621,1508180974,0xfe55e8da4fd807d2a25518a2ff9bab09fa34e763ea0d...,1.0,0xe7545f76ecbc8435d348c79db53ea6e6693578ff,0xcd68b4f73fd20e6d50dc1dbec229addcba50554a,0x0000000000000000000000000000000000000000,0xb97048628db6b661d4c2aa833e95dbe1a905b280,0.370104,0.060278,62.320000,10.150000,168.385206,0.001,0.013281,290000,4000000000,1692,1,190000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5344144,9999990,1588598385,0xe25d774ce7eedbdbdcd6d21dc478c1ea1f35c2a1ac54...,1.0,0x79147f4a41d558d2340034f7ef41525edd2966d0,0x4aa08b97955ddd3829ad17eb930c1c8903cf20ea,0x0000000000000000000000000000000000000000,0xd947b0ceab2a8885866b9a04a06ae99de852a3d4,0.254160,0.254160,1632.056635,1632.056635,6421.371829,0.001,0.010026,400000,11000000000,1588598108158596,27,100000
5344145,9999997,1588598492,0xf049d457a35e6ab08430662240f133e05d4d74e920f2...,1.0,0x7d825166cd469d49f480ff208183fa7546d2825b,0x12d6956c30b8309c6396fb47ac500dc8e1f793fc,0x0000000000000000000000000000000000000000,0x4a220e6096b25eadb88358cb44068a3248254675,4.105321,0.344795,175.026394,14.700000,42.634033,0.001,0.007917,400000,11000000000,1588609310996,94,10000
5344146,9999997,1588598492,0xeadc59e8407b1566ae755a6a980cc971f12fed7a9c6d...,1.0,0xaec346cf77ccfad030f4e83db29e8624c65ed295,0x12d6956c30b8309c6396fb47ac500dc8e1f793fc,0x0000000000000000000000000000000000000000,0x4a220e6096b25eadb88358cb44068a3248254675,0.211108,0.211108,9.000183,9.000183,42.633137,0.001,0.011663,400000,11000000000,51334745949,93,10000
5344147,9999998,1588598524,0xe63bd29f97a3fd9035a9581ed4ca6fecb310cceee4d7...,1.0,0xe706b6071a5459fa34f7c85d938edd1912a4cf56,0x9fee99fa1eb0c0e7ddbba00ce764ae1fe9bd2684,0x0000000000000000000000000000000000000000,0xe5caef4af8780e59df925470b050fb23c43ca68c,0.887616,0.887616,14482.000000,14482.000000,16315.609143,0.000,0.004298,400000,11000000000,351037383,119272554,100000


In [7]:
def merge_trades_with_daily_usd_price(trades, price_file_csv="data/EtherDollarPrice.csv"):
    ether_dollar = pd.read_csv(price_file_csv)
    ether_dollar.columns = ["date", "timestamp", "dollar"]
    ether_dollar['date'] = pd.to_datetime(ether_dollar['date'], format='%m/%d/%Y')

    # Add timestamp of date to trades for merging
    min_trades_timestamp = trades['timestamp'].min()
    max_trades_timestamp = trades['timestamp'].max()

    # Get greatest Dollar timestamp that is smaller-equal than the smallest trades timestamp
    min_dollar_timestamp = ether_dollar[ether_dollar['timestamp'] <= min_trades_timestamp].sort_values('timestamp').iloc[-1]['timestamp']

    # Get smallest Dollar timestamp that is greater-equal than the greatest trades timestamp
    max_dollar_timestamp = ether_dollar[ether_dollar['timestamp'] >= max_trades_timestamp].iloc[0]['timestamp']

    # Get left sides of intervals
    intervals_left = ether_dollar[(ether_dollar['timestamp'] >= min_dollar_timestamp) & 
                                  (ether_dollar['timestamp'] <= max_dollar_timestamp)]['timestamp']

    # Cut IDEX timestamps based on intervals
    trades['cut'] = pd.cut(trades['timestamp'], bins=intervals_left, labels=intervals_left[:-1], 
                           include_lowest=True, right=False)
    trades['cut'] = trades['cut'].astype(float)

    # Merge buy eth trades with eth-dollar price
    trades_buyeth = trades[trades['tokenBuy'] == global_ether_id]
    trades_buyeth = pd.merge(trades_buyeth, ether_dollar[['timestamp', 'dollar', 'date']], 
                             left_on='cut', right_on='timestamp', suffixes=('', '_y'))
    trades_buyeth = trades_buyeth.rename(columns={'dollar': 'eth_price'})
    trades_buyeth = trades_buyeth[['date', 'cut', 'blockNumber', 'timestamp', 'transactionHash',
                                   'maker', 'taker', 'tokenBuy', 'tokenSell', 'amountBoughtReal',
                                   'amountSoldReal', 'price', 'feeMake', 'feeTake', 'eth_price']]
    trades_buyeth = trades_buyeth.rename(columns={
        'maker': 'eth_buyer', 'taker': 'eth_seller', 'tokenBuy': 'ether', 'tokenSell': 'token',
        'amountBoughtReal': 'trade_amount_eth', 'amountSoldReal': 'trade_amount_token',
        'feeMake': 'fee_eth_buyer', 'feeTake': 'fee_eth_seller'
    })
    trades_buyeth['trade_amount_dollar'] = trades_buyeth['trade_amount_eth'] * trades_buyeth['eth_price']
    trades_buyeth['token_price_in_eth'] = 1 / trades_buyeth['price']

    # Merge sell eth trades with eth-dollar price
    trades_selleth = trades[trades['tokenSell'] == global_ether_id]
    trades_selleth = pd.merge(trades_selleth, ether_dollar[['timestamp', 'dollar', 'date']], 
                              left_on='cut', right_on='timestamp', suffixes=('', '_y'))
    trades_selleth = trades_selleth.rename(columns={'dollar': 'eth_price'})
    trades_selleth = trades_selleth[['date', 'cut', 'blockNumber', 'timestamp', 'transactionHash',
                                     'maker', 'taker', 'tokenSell', 'tokenBuy', 'amountSoldReal',
                                     'amountBoughtReal', 'price', 'feeMake', 'feeTake', 'eth_price']]
    trades_selleth = trades_selleth.rename(columns={
        'taker': 'eth_buyer', 'maker': 'eth_seller', 'tokenSell': 'ether', 'tokenBuy': 'token',
        'amountSoldReal': 'trade_amount_eth', 'amountBoughtReal': 'trade_amount_token',
        'feeTake': 'fee_eth_buyer', 'feeMake': 'fee_eth_seller'
    })
    trades_selleth['trade_amount_dollar'] = trades_selleth['trade_amount_eth'] * trades_selleth['eth_price']
    trades_selleth['token_price_in_eth'] = trades_selleth['price']

    # Concatenate and sort
    trades_eth = pd.concat([trades_buyeth, trades_selleth]).sort_values('blockNumber')

    return trades_eth

In [8]:
def merge_EtherDelta_trades_with_daily_usd_price(trades, price_file_csv="data/EtherDollarPrice.csv"):
    ether_dollar = pd.read_csv(price_file_csv)
    ether_dollar.columns = ["date", "timestamp", "dollar"]
    ether_dollar['date'] = pd.to_datetime(ether_dollar['date'], format="%m/%d/%Y")

    # Add timestamp of date to trades for merging
    min_trades_timestamp = trades['timestamp'].min()
    max_trades_timestamp = trades['timestamp'].max()

    # Get greatest Dollar timestamp that is smaller-equal than the smallest trades timestamp
    min_dollar_timestamp = ether_dollar[ether_dollar['timestamp'] <= min_trades_timestamp].sort_values('timestamp').iloc[-1]['timestamp']

    # Get smallest Dollar timestamp that is greater-equal than the greatest trades timestamp
    max_dollar_timestamp = ether_dollar[ether_dollar['timestamp'] >= max_trades_timestamp].iloc[0]['timestamp']

    # Get left sides of intervals
    intervals_left = ether_dollar[(ether_dollar['timestamp'] >= min_dollar_timestamp) & 
                                  (ether_dollar['timestamp'] <= max_dollar_timestamp)]['timestamp']

    # Cut IDEX timestamps based on intervals
    trades['cut'] = pd.cut(trades['timestamp'], bins=intervals_left, labels=intervals_left[:-1], include_lowest=True, right=False)
    trades['cut'] = trades['cut'].astype(float)

    # Merge buy eth trades with eth-dollar price
    trades_buyeth = trades[trades['tokenBuy'] == global_ether_id].merge(
        ether_dollar[['timestamp', 'dollar', 'date']].rename(columns={'dollar': 'eth_price'}),
        left_on='cut', right_on='timestamp', suffixes=('', '_y')
    )
    trades_buyeth = trades_buyeth.rename(columns={
        'maker': 'eth_buyer', 'taker': 'eth_seller', 'tokenBuy': 'ether', 'tokenSell': 'token',
        'amountBoughtReal': 'trade_amount_eth', 'amountSoldReal': 'trade_amount_token'
    })
    trades_buyeth['trade_amount_dollar'] = trades_buyeth['trade_amount_eth'] * trades_buyeth['eth_price']
    trades_buyeth['token_price_in_eth'] = 1 / trades_buyeth['price']

    # Merge sell eth trades with eth-dollar price
    trades_selleth = trades[trades['tokenSell'] == global_ether_id].merge(
        ether_dollar[['timestamp', 'dollar', 'date']].rename(columns={'dollar': 'eth_price'}),
        left_on='cut', right_on='timestamp', suffixes=('', '_y')
    )
    trades_selleth = trades_selleth.rename(columns={
        'taker': 'eth_buyer', 'maker': 'eth_seller', 'tokenSell': 'ether', 'tokenBuy': 'token',
        'amountSoldReal': 'trade_amount_eth', 'amountBoughtReal': 'trade_amount_token'
    })
    trades_selleth['trade_amount_dollar'] = trades_selleth['trade_amount_eth'] * trades_selleth['eth_price']
    trades_selleth['token_price_in_eth'] = trades_selleth['price']

    # Concatenate and sort
    trades_eth = pd.concat([trades_buyeth, trades_selleth]).sort_values('blockNumber')

    # Select and reorder columns
    columns = ['date', 'cut', 'blockNumber', 'timestamp', 'transactionHash', 'eth_buyer', 'eth_seller', 
               'ether', 'token', 'trade_amount_eth', 'trade_amount_dollar', 'trade_amount_token', 'token_price_in_eth']
    trades_eth = trades_eth[columns]

    return trades_eth

In [9]:
if 'EtherDelta' in data:
    ___trades = merge_EtherDelta_trades_with_daily_usd_price(__trades)
else:
    ___trades = merge_trades_with_daily_usd_price(__trades)
___trades

,date,cut,blockNumber,timestamp,transactionHash,eth_buyer,eth_seller,ether,token,trade_amount_eth,trade_amount_token,price,fee_eth_buyer,fee_eth_seller,eth_price,trade_amount_dollar,token_price_in_eth
0,2017-10-16,1.508112e+09,4371302,1508159416,0xdcddd3aee6ccc5df4a59d46cb3742509dd5ab2cc22ed...,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,1.500000,1153.846154,769.230769,0.001000,0.003133,334.23,501.345000,0.001300
1,2017-10-16,1.508112e+09,4371451,1508162021,0xba3fc442e5df6b53d011983511644b2582f99b64dd19...,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,7.500000,5000.000000,666.666667,0.001000,0.002181,334.23,2506.725000,0.001500
2,2017-10-16,1.508112e+09,4371467,1508162270,0x54a5606d69477b310e74347696b854bd64ae44124d4b...,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,7.750000,5000.000000,645.161290,0.001000,0.002175,334.23,2590.282500,0.001550
0,2017-10-16,1.508112e+09,4371574,1508164311,0xd18c4cc92549568e00f0f22a829c98bb16aad45f336d...,0x55475d4434495a93521afb24e67483f0d05701ba,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,0.420000,350.000000,0.001200,0.005238,0.001000,334.23,140.376600,0.001200
3,2017-10-16,1.508112e+09,4372621,1508180974,0xfe55e8da4fd807d2a25518a2ff9bab09fa34e763ea0d...,0xe7545f76ecbc8435d348c79db53ea6e6693578ff,0xcd68b4f73fd20e6d50dc1dbec229addcba50554a,0x0000000000000000000000000000000000000000,0xb97048628db6b661d4c2aa833e95dbe1a905b280,0.060278,10.150000,168.385206,0.001000,0.013281,334.23,20.146868,0.005939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2932855,2020-05-04,1.588550e+09,9999990,1588598385,0xe25d774ce7eedbdbdcd6d21dc478c1ea1f35c2a1ac54...,0x79147f4a41d558d2340034f7ef41525edd2966d0,0x4aa08b97955ddd3829ad17eb930c1c8903cf20ea,0x0000000000000000000000000000000000000000,0xd947b0ceab2a8885866b9a04a06ae99de852a3d4,0.254160,1632.056635,6421.371829,0.001000,0.010026,206.94,52.595895,0.000156
2932856,2020-05-04,1.588550e+09,9999997,1588598492,0xf049d457a35e6ab08430662240f133e05d4d74e920f2...,0x7d825166cd469d49f480ff208183fa7546d2825b,0x12d6956c30b8309c6396fb47ac500dc8e1f793fc,0x0000000000000000000000000000000000000000,0x4a220e6096b25eadb88358cb44068a3248254675,0.344795,14.700000,42.634033,0.001000,0.007917,206.94,71.351870,0.023455
2932857,2020-05-04,1.588550e+09,9999997,1588598492,0xeadc59e8407b1566ae755a6a980cc971f12fed7a9c6d...,0xaec346cf77ccfad030f4e83db29e8624c65ed295,0x12d6956c30b8309c6396fb47ac500dc8e1f793fc,0x0000000000000000000000000000000000000000,0x4a220e6096b25eadb88358cb44068a3248254675,0.211108,9.000183,42.633137,0.001000,0.011663,206.94,43.686625,0.023456
2932859,2020-05-04,1.588550e+09,9999998,1588598524,0xf0840595854ee8a4d69d1eac18e53ef7bb2de555bb92...,0x5fb0ce08f60de8b87917f28c87963aa3ebfa4418,0xf59ade7712797193264ef62b0a412ac1c94fc3a9,0x0000000000000000000000000000000000000000,0xb9eefc4b0d472a44be93970254df4f4016569d27,1.247255,12206.446467,9786.651008,0.000000,0.001636,206.94,258.106888,0.000102


## Selftrades

In [10]:
def filter_self_trades(trades, save=True, folder="output", filename="self_trades"):
    self_trades = trades[trades['eth_buyer'] == trades['eth_seller']]
    non_self_trades = trades[trades['eth_buyer'] != trades['eth_seller']]
    print(f"Info: filtered {len(self_trades)} self-trades. {len(non_self_trades)} non-self-trades remaining.")
    if save:
        filename = filename.split(".")[0]
        self_trades.to_csv(f"{folder}/{filename}.csv", index=False)
    return {'self_trades': self_trades, 'non_self_trades': non_self_trades}

l = filter_self_trades(___trades, True, output_folder)
self_trades = l['self_trades']

Info: filtered 28193 self-trades. 5312344 non-self-trades remaining.


In [11]:
def summarize_self_trades(self_trades, save=True, folder="output", filename="self_trades_summary"):
    summary = self_trades.groupby(['eth_buyer', 'token']).agg({
        'trade_amount_eth': 'sum',
        'trade_amount_dollar': 'sum',
        'trade_amount_token': 'sum',
        'date': ['min', 'max'],
        'transactionHash': 'count'
    })
    summary = summary.sort_values([('date', 'min')]).reset_index()
    summary.columns = ['trader', 'token', 'tx_sum_eth', 'tx_sum_dollar', 'tx_sum_token', 'start_date', 'end_date', 'tx_count']
    if save:
        filename = filename.split(".")[0]
        summary.to_csv(f"{folder}/{filename}.csv", index=False)
    return summary

summarize_self_trades(self_trades, True, output_folder)

,trader,token,tx_sum_eth,tx_sum_dollar,tx_sum_token,start_date,end_date,tx_count
0,0x034767f3c519f361c5ecf46ebfc08981c629d381,0xadc46ff5434910bd17b24ffb429e585223287d7f,7.560000,2293.465300,1.010000e-06,2017-10-16,2017-10-20,2
1,0xe7545f76ecbc8435d348c79db53ea6e6693578ff,0x0abdace70d3790235af448c88547603b945604ea,366.502391,155878.087363,2.613730e+06,2017-10-17,2018-01-26,214
2,0xe7545f76ecbc8435d348c79db53ea6e6693578ff,0x41e5560054824ea6b0732e656e3ad64e20e94e45,527.492668,210117.638986,5.559056e+05,2017-10-17,2018-01-26,246
3,0xe7545f76ecbc8435d348c79db53ea6e6693578ff,0x27695e09149adc738a978e9a678f99e4c39e9eb9,79.028733,25680.482655,5.095929e-05,2017-10-17,2018-01-02,107
4,0xe7545f76ecbc8435d348c79db53ea6e6693578ff,0x86fa049857e0209aa7d9e616f7eb3b3b78ecfdb0,1039.030914,428330.610651,2.614683e+05,2017-10-17,2018-01-29,262
...,...,...,...,...,...,...,...,...
461,0x5ab9d116a53ef41063e3eae26a7ebe736720e9ba,0x574f84108a98c575794f75483d801d1d5dc861a5,0.400000,54.696000,4.395604e+03,2020-03-23,2020-03-23,1
462,0x553ec10c3582fcc7d2302f7450c2a7422f58800c,0x0f02e27745e3b6e9e1310d19469e2b5d7b5ec99a,0.754051,103.406211,1.083501e+05,2020-03-26,2020-04-04,4
463,0xbb732b094014227827edb0930eb2b9b3c958abc5,0x0f02e27745e3b6e9e1310d19469e2b5d7b5ec99a,0.541145,76.544977,6.937742e+04,2020-04-03,2020-04-03,3
464,0xbb732b094014227827edb0930eb2b9b3c958abc5,0x0ba45a8b5d5575935b8158a88c631e9f9c95a2e5,0.853667,145.960049,3.617235e+01,2020-04-17,2020-04-17,1


In [12]:
non_self_trades = l['non_self_trades']

In [13]:
def add_trader_hashes(trades, trader_hashes):

    if trader_hashes.empty:
        trader_addresses = sorted(set(trades['eth_buyer']).union(set(trades['eth_seller'])))
        trader_hashes = pd.DataFrame({
            'trader_address': trader_addresses,
            'trader_id': range(1, len(trader_addresses) + 1)
        })
    else:
        additional_traders = set(trades['eth_buyer']).union(set(trades['eth_seller'])) - set(trader_hashes['trader_address'])
        if additional_traders:
            n_old = len(trader_hashes)
            new_traders = pd.DataFrame({
                'trader_address': sorted(additional_traders),
                'trader_id': range(n_old + 1, n_old + len(additional_traders) + 1)
            })
            trader_hashes = pd.concat([trader_hashes, new_traders], ignore_index=True)
    
    trades = trades.merge(trader_hashes.rename(columns={'trader_address': 'eth_buyer', 'trader_id': 'eth_buyer_id'}), on='eth_buyer', how='left')
    trades = trades.merge(trader_hashes.rename(columns={'trader_address': 'eth_seller', 'trader_id': 'eth_seller_id'}), on='eth_seller', how='left')
    trades = trades.sort_values('timestamp')
    return trades, trader_hashes

_non_self_trades, trader_hashes = add_trader_hashes(non_self_trades, global_trader_hashes)
global_trader_hashes = trader_hashes
_non_self_trades

,date,cut,blockNumber,timestamp,transactionHash,eth_buyer,eth_seller,ether,token,trade_amount_eth,trade_amount_token,price,fee_eth_buyer,fee_eth_seller,eth_price,trade_amount_dollar,token_price_in_eth,eth_buyer_id,eth_seller_id
0,2017-10-16,1.508112e+09,4371302,1508159416,0xdcddd3aee6ccc5df4a59d46cb3742509dd5ab2cc22ed...,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,1.500000,1153.846154,769.230769,0.001000,0.003133,334.23,501.345000,0.001300,105322,169664
1,2017-10-16,1.508112e+09,4371451,1508162021,0xba3fc442e5df6b53d011983511644b2582f99b64dd19...,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,7.500000,5000.000000,666.666667,0.001000,0.002181,334.23,2506.725000,0.001500,105322,169664
2,2017-10-16,1.508112e+09,4371467,1508162270,0x54a5606d69477b310e74347696b854bd64ae44124d4b...,0x6c28846676326bf6820e3b10c59fcd3717c42081,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,7.750000,5000.000000,645.161290,0.001000,0.002175,334.23,2590.282500,0.001550,105322,169664
3,2017-10-16,1.508112e+09,4371574,1508164311,0xd18c4cc92549568e00f0f22a829c98bb16aad45f336d...,0x55475d4434495a93521afb24e67483f0d05701ba,0xad7f84324d7e4e042ac6bce6f29c2f9b4f536cc6,0x0000000000000000000000000000000000000000,0x514910771af9ca656af840dff83e8264ecf986ca,0.420000,350.000000,0.001200,0.005238,0.001000,334.23,140.376600,0.001200,82986,169664
4,2017-10-16,1.508112e+09,4372621,1508180974,0xfe55e8da4fd807d2a25518a2ff9bab09fa34e763ea0d...,0xe7545f76ecbc8435d348c79db53ea6e6693578ff,0xcd68b4f73fd20e6d50dc1dbec229addcba50554a,0x0000000000000000000000000000000000000000,0xb97048628db6b661d4c2aa833e95dbe1a905b280,0.060278,10.150000,168.385206,0.001000,0.013281,334.23,20.146868,0.005939,225865,200680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5312339,2020-05-04,1.588550e+09,9999990,1588598385,0xe25d774ce7eedbdbdcd6d21dc478c1ea1f35c2a1ac54...,0x79147f4a41d558d2340034f7ef41525edd2966d0,0x4aa08b97955ddd3829ad17eb930c1c8903cf20ea,0x0000000000000000000000000000000000000000,0xd947b0ceab2a8885866b9a04a06ae99de852a3d4,0.254160,1632.056635,6421.371829,0.001000,0.010026,206.94,52.595895,0.000156,118069,72578
5312340,2020-05-04,1.588550e+09,9999997,1588598492,0xf049d457a35e6ab08430662240f133e05d4d74e920f2...,0x7d825166cd469d49f480ff208183fa7546d2825b,0x12d6956c30b8309c6396fb47ac500dc8e1f793fc,0x0000000000000000000000000000000000000000,0x4a220e6096b25eadb88358cb44068a3248254675,0.344795,14.700000,42.634033,0.001000,0.007917,206.94,71.351870,0.023455,122428,18711
5312341,2020-05-04,1.588550e+09,9999997,1588598492,0xeadc59e8407b1566ae755a6a980cc971f12fed7a9c6d...,0xaec346cf77ccfad030f4e83db29e8624c65ed295,0x12d6956c30b8309c6396fb47ac500dc8e1f793fc,0x0000000000000000000000000000000000000000,0x4a220e6096b25eadb88358cb44068a3248254675,0.211108,9.000183,42.633137,0.001000,0.011663,206.94,43.686625,0.023456,170879,18711
5312342,2020-05-04,1.588550e+09,9999998,1588598524,0xf0840595854ee8a4d69d1eac18e53ef7bb2de555bb92...,0x5fb0ce08f60de8b87917f28c87963aa3ebfa4418,0xf59ade7712797193264ef62b0a412ac1c94fc3a9,0x0000000000000000000000000000000000000000,0xb9eefc4b0d472a44be93970254df4f4016569d27,1.247255,12206.446467,9786.651008,0.000000,0.001636,206.94,258.106888,0.000102,93087,239684


## SCC

In [14]:
import networkx as nx
from tqdm import tqdm
import hashlib
import os
from collections import Counter

In [15]:
def detect_scc_for_tokens_layered_old(trades, global_scc_traders_map, save=True, folder="output", filename="scc"):

    # convert trades to polars
    trades = pl.from_pandas(trades)

    # Get unique tokens
    token_vector = trades['token'].unique()
    results = []
    
    # Iterate through each token
    for token in tqdm(token_vector, desc="Processing tokens"):
        token_trades = trades.filter(pl.col('token') == token)

        edges = Counter((row['eth_buyer_id'], row['eth_seller_id']) for row in token_trades.select(['eth_buyer_id', 'eth_seller_id']).to_dicts())
        
        g = nx.DiGraph()
        for (u, v), w in edges.items(): 
            g.add_edge(u, v, weight=w)

        while g.number_of_nodes() > 0:
            # Find strongly connected components
            sccs = [list(comp) for comp in nx.strongly_connected_components(g) if len(comp) > 1]

            if len(sccs) == 0:
                break
            
            for scc in sccs:
                sorted_members = sorted(scc)
                c_hash = hashlib.md5(','.join(str(sorted_members)).encode()).hexdigest()
                global_scc_traders_map[c_hash] = sorted_members
                results.append(c_hash)

            # Decrease weights by one
            zero_weight_edges = []
            for u, v, d in g.edges(data=True):
                d['weight'] -= 1
                if d['weight'] == 0:
                    zero_weight_edges.append((u, v))
            
            # Remove zero-weight edges and isolated nodes
            g.remove_edges_from(zero_weight_edges)
            g.remove_nodes_from(list(nx.isolates(g)))

    
    # Create DataFrame for results
    scc_df = pd.DataFrame({'scc_hash': results})
    scc_summary = scc_df.groupby('scc_hash').size().reset_index(name='occurrence')
    scc_summary['num_traders'] = scc_summary['scc_hash'].apply(lambda x: len(global_scc_traders_map[x]))
    
    if save:
        # Save the results
        scc_summary.to_csv(f"{folder}/{filename}.csv", index=False)
        
        mapping = pd.DataFrame([(k, v) for k, values in global_scc_traders_map.items() for v in values], columns=['hash', 'trader_id'])
        mapping.to_csv(f"{folder}/{filename}-mapping.csv", index=False)
    
    return scc_summary

scc_dt = detect_scc_for_tokens_layered_old(_non_self_trades, global_scc_traders_map, save=True, folder=output_folder)
scc_dt

Processing tokens: 100%|██████████| 1199/1199 [00:29<00:00, 39.97it/s]


,scc_hash,occurrence,num_traders
0,0007cd79282adc142249de85dd8b085e,3,2
1,00084ead92a640d9c85a2e6dd8827085,4,15
2,000cf8a16a1e355118394556b94f51d4,1,163
3,001ff671606180b9c23719525107a2ed,1,4
4,0026d65eafad54ed0c76e9cf36eb5e88,1,2
...,...,...,...
6486,ffd7f360679b6bebb855013c9b939365,2,6
6487,ffdcbdcd4fd77ccad2679eb62eeaab37,3,5
6488,ffe813c4b534b6c54ce23f4b5e49b5e5,1,89
6489,ffea896d260790cfab15a9f9aa99f624,1,3


In [16]:
def get_relevant_scc_by_threshold(scc_df, threshold):
    relevant_sccs = scc_df[scc_df['occurrence'] >= threshold]
    print(f"Info: Determined {len(relevant_sccs)} unique SCCs to be relevant at threshold {threshold}")
    print(f"Info: Minimum occurrence is {relevant_sccs['occurrence'].min()}")
    return relevant_sccs['scc_hash'].tolist()

relevant_scc_ids = get_relevant_scc_by_threshold(scc_dt, scc_threshold_rank)
# relevant_scc_ids

Info: Determined 199 unique SCCs to be relevant at threshold 100
Info: Minimum occurrence is 100


## Analyse SCCs Trades

In [17]:
def detect_label_wash_trades(df: pl.DataFrame, margin: float = 0.1) -> pl.DataFrame:
    buyers = df['buyer'].to_list()
    sellers = df['seller'].to_list()
    amounts = df['amount'].to_list()

    balance_map = {}
    trade_amounts = []

    for buyer, seller, amount in zip(buyers, sellers, amounts):
        trade_amounts.append(amount)
        balance_map[buyer] = balance_map.get(buyer, 0) + amount
        balance_map[seller] = balance_map.get(seller, 0) - amount

    for idx in range(len(df) - 1, 0, -1):
        balances = np.array(list(balance_map.values()))
        mean_trade_vol = np.mean(trade_amounts)

        if mean_trade_vol == 0:
            mean_trade_vol = 1

        balances = np.abs(balances / mean_trade_vol)

        if np.all(balances <= margin):
            wash_indices = list(range(idx + 1))
            df = df.with_columns(
                pl.when(pl.col('wash_label').is_null())
                .then(pl.lit(False))
                .otherwise(pl.col('wash_label'))
                .alias('wash_label')
            )
            df = df.with_columns(
                pl.when(pl.arange(0, len(df)).is_in(wash_indices))
                .then(pl.lit(True))
                .otherwise(pl.col('wash_label'))
                .alias('wash_label')
            )
            return df

        amount = amounts[idx]
        trade_amounts.pop()

        balance_map[buyers[idx]] -= amount
        balance_map[sellers[idx]] += amount

    return df

In [18]:
def detect_and_label_wash_trades_for_scc_using_multiple_passes(
    trades, relevant_scc, window_sizes_in_seconds, window_start=None, 
    ether=True, margin=0.1, save=True, folder="output", 
    filename="wash_trades_multiple_windows"
):   
    print(f"Starting wash trade labeling with {len(window_sizes_in_seconds)} passes.")
    
    checked_trades_hashes = []
    checked_trades_labels = []

    # Convert to polars DataFrame
    trades = pl.from_pandas(trades)

    trades = trades.with_columns(pl.lit(None).alias("wash_label"))

    # if window start is not given, take start of first day of given trades
    if window_start is None:
        window_start = trades['cut'].min()

    wash_trades = {}
    
    for window_size in window_sizes_in_seconds:

        # breaks from start to last timestamp (incl.), by given steps in seconds
        intervals = np.arange(window_start, trades['timestamp'].max(), window_size)
        
        for scc_id in tqdm(relevant_scc, desc=f"Processing SCCs for window size {window_size}"):

            scc_traders = global_scc_traders_map[scc_id]
            
            # Filter trades for the relevant SCC
            scc_trades = trades.filter(
                (pl.col("eth_seller_id").is_in(scc_traders)) & 
                (pl.col("eth_buyer_id").is_in(scc_traders)) & 
                ((pl.col("wash_label") == False) | (pl.col("wash_label").is_null()))
            ).sort("cut")
            
            if len(scc_trades) == 0:
                # wash_trades[scc_id] = {str(window_size): []}
                # wash_trades.setdefault(scc_id, {}).setdefault(str(window_size), {})
                continue
            
            # label these trades as FALSE in original trade set to indicate they have been checked
            trades = trades.with_columns(
                pl.when(pl.col("transactionHash").is_in(scc_trades["transactionHash"]))
                .then(False)
                .otherwise(pl.col("wash_label"))
                .alias("wash_label")
            )

            # Prepare trades for processing
            temp_trades = scc_trades.select([
                "transactionHash", "token", "date", "timestamp", "trade_amount_dollar", "wash_label",
                pl.col("eth_buyer" if ether else "eth_seller").alias("buyer"),
                pl.col("eth_seller" if ether else "eth_buyer").alias("seller"),
                pl.col("trade_amount_eth" if ether else "trade_amount_token").alias("amount")
            ])


            # Process trades in time windows
            temp_trades = temp_trades.with_columns(
                pl.col("timestamp").cut(intervals, left_closed=True).alias("interval")
            )

            for names, data in temp_trades.group_by(['token', 'interval']):

                detected_wash_trades = detect_label_wash_trades(data, margin)

                # copy tx_hash and label to checked_trades
                checked_trades_hashes.extend(detected_wash_trades['transactionHash'])
                checked_trades_labels.extend(detected_wash_trades['wash_label'])
 
                wash_trades.setdefault(scc_id, {}).setdefault(str(window_size), {})['.'.join(names)] = detected_wash_trades


            # update trades with checked_trades
            # join checked_trades to trades and replace with wash_label from checked_trades
            checked_trades_df = pl.DataFrame(
                {"transactionHash": checked_trades_hashes,"wash_label": checked_trades_labels}, 
                schema={"transactionHash": pl.Utf8, "wash_label": pl.Boolean}
            )
            try:
                tx_hash_true_list = checked_trades_df.filter(checked_trades_df['wash_label'] == True)['transactionHash'].to_list()
            except Exception as e:
                tx_hash_true_list = []

            trades = (
                trades
                .with_columns(
                    pl.when(pl.col("transactionHash").is_in(tx_hash_true_list))
                    .then(True)
                    .otherwise(pl.col("wash_label"))
                    .alias("wash_label")
                )
            )


            checked_trades_hashes = []
            checked_trades_labels = []

    if save:
        # Save results
        filename = filename.split('.')[0]
        trades.write_csv(os.path.join(folder, "trades_labeled.csv"))
        # np.save(os.path.join(folder, f"{filename}.npy"), wash_trades)
    
    return wash_trades, trades



wash_trades, trades_labeled = detect_and_label_wash_trades_for_scc_using_multiple_passes(
    _non_self_trades, relevant_scc_ids, wash_window_sizes_seconds, ether=wash_trade_detection_ether,
    margin=wash_trade_detection_margin, save=True, folder=output_folder)

Starting wash trade labeling with 3 passes.


Processing SCCs for window size 604800: 100%|██████████| 199/199 [01:17<00:00,  2.56it/s]


In [19]:
def get_summary_of_wash_trades_per_scc_and_timewindow(wash_trades, window_size_name, multiple_passes=False, 
                                                      save=True, folder="output", filename="wash_trades_summary"):
    print("Info: producing wash trading summary...")

    rows = []

    if multiple_passes:

        for scc in wash_trades.keys():

            for window_size in wash_trades[scc].keys():

                for w in wash_trades[scc][window_size].keys():

                    temp = w.split('.')
                    token = temp[0]
                    window = temp[1]

                    wash_trades_scc_window = wash_trades[scc][window_size][w].with_columns(
                        pl.col("wash_label").cast(pl.Boolean) # cast wash_label to bool
                    )

                    num_wash = (wash_trades_scc_window['wash_label'] == True).sum()
                    num_all = len(wash_trades_scc_window)
                    amount_wash = wash_trades_scc_window.filter(pl.col.wash_label == True)['amount'].sum()
                    amount_all = wash_trades_scc_window['amount'].sum()
                    amount_dollar_wash = wash_trades_scc_window.filter(pl.col.wash_label == True)['trade_amount_dollar'].sum()
                    amount_dollar_all = wash_trades_scc_window['trade_amount_dollar'].sum()
                    new_row = {'scc_hash': scc, 'token': token, 'window_size': window_size, 
                            'time': window, 'num_wash_trades': num_wash, 'num_trades': num_all, 
                            'total_amount_wash': amount_wash, 'total_amount': amount_all, 
                            'total_amount_dollar_wash': amount_dollar_wash, 
                            'total_amount_dollar': amount_dollar_all}

                    rows.append(new_row)

    
    else:
        for scc in wash_trades.keys():
            for w in wash_trades[scc].keys():

                temp = w.split('.')
                token = temp[0]
                window = temp[1]

                wash_trades_scc_window = wash_trades[scc][w].with_columns(
                    pl.col("wash_label").cast(pl.Boolean) # cast wash_label to bool
                )

                num_wash = (wash_trades_scc_window['wash_label'] == True).sum()
                num_all = len(wash_trades_scc_window)
                amount_wash = wash_trades_scc_window.filter(pl.col.wash_label == True)['amount'].sum()
                amount_all = wash_trades_scc_window['amount'].sum()
                amount_dollar_wash = wash_trades_scc_window.filter(pl.col.wash_label == True)['trade_amount_dollar'].sum()
                amount_dollar_all = wash_trades_scc_window['trade_amount_dollar'].sum()
                new_row = {'scc_hash': scc, 'token': token, #'window_size': window_size, 
                        'time': window, 'num_wash_trades': num_wash, 'num_trades': num_all, 
                        'total_amount_wash': amount_wash, 'total_amount': amount_all, 
                        'total_amount_dollar_wash': amount_dollar_wash, 
                        'total_amount_dollar': amount_dollar_all}

                rows.append(new_row)
    
    wash_trades_dt = pl.DataFrame(rows)


    if save:
        filename = os.path.splitext(filename)[0]
        wash_trades_dt.write_csv(os.path.join(folder, f"{filename}_{window_size_name}.csv"))
    
    return wash_trades_dt


wash_trades_multiple_passes_summary = get_summary_of_wash_trades_per_scc_and_timewindow(wash_trades, "multiple_windows", multiple_passes=True, save=True, folder=output_folder)
wash_trades_multiple_passes_summary

Info: producing wash trading summary...


scc_hash,token,window_size,time,num_wash_trades,num_trades,total_amount_wash,total_amount,total_amount_dollar_wash,total_amount_dollar
str,str,str,str,i64,i64,f64,f64,f64,f64
"""00ea309c6af27f…","""0x4e15361fd6b4…","""3600""","""[1556211600, 1…",0,2,0.0,140879.416722,0.0,1265.691646
"""00ea309c6af27f…","""0x4e15361fd6b4…","""3600""","""[1556661600, 1…",2,7,131193.239423,361430.988032,1212.862838,3336.468627
"""00ea309c6af27f…","""0x4e15361fd6b4…","""3600""","""[1556798400, 1…",0,5,0.0,198637.425628,0.0,1926.448759
"""00ea309c6af27f…","""0x4e15361fd6b4…","""3600""","""[1555412400, 1…",0,3,0.0,174505.940449,0.0,1871.109338
"""00ea309c6af27f…","""0x4e15361fd6b4…","""3600""","""[1556204400, 1…",0,1,0.0,63132.899788,0.0,555.766662
…,…,…,…,…,…,…,…,…,…
"""ff62013cf31218…","""0xd0352a019e9a…","""604800""","""[1561334400, 1…",0,142,0.0,20147.297437,0.0,33182.430159
"""ff62013cf31218…","""0xd0352a019e9a…","""604800""","""[1560729600, 1…",0,1258,0.0,339640.671692,0.0,557769.300182
"""ff62013cf31218…","""0xd0352a019e9a…","""604800""","""[1557705600, 1…",0,4,0.0,1065.616563,0.0,1006.175287


In [20]:
import json

def get_address_clusters(trades, scc_ids, save=True, folder="output", filename="address_clusters"):
    address_clusters = {}

    # for each SCC
    for scc_id in scc_ids:
        scc_traders = global_scc_traders_map.get(scc_id, [])
        trader_addresses = global_trader_hashes[global_trader_hashes['trader_id'].isin(scc_traders)]['trader_address'].tolist()
        address_clusters[str(scc_id)] = trader_addresses

    # save file
    if save:
        filename = filename.split('.')[0]
        with open(f"{folder}/{filename}.json", "w") as outfile:
            json.dump(address_clusters, outfile)

    return address_clusters

get_address_clusters(wash_trades, relevant_scc_ids, save=True, folder=output_folder)

{'00ea309c6af27fbcbde69a4548e04b2b': ['0x1042320137d4e923711e33d8f758e512a235ef8e',
  '0x8f3b9cb5efbd1837cb7fc0be776b3f2372d7fa67'],
 '033894b02894bdbf2f2960d85927246b': ['0x900873d1378c26a71bdd499553f470a97ff752a4',
  '0xbd58cdf2266552d39ad8621793bd60c5b9f97e38',
  '0xcf4de24bda6193ba64fd5ba07de8e2ad925fd969',
  '0xe706b6071a5459fa34f7c85d938edd1912a4cf56'],
 '03cb74651ae5e4888e1681d788b53e07': ['0x3863293562cf39f441e8bb2f596d43af397b1ce1',
  '0xf9773dd8be1d0aeb2cc91318d305e9973ec6ee37'],
 '0407083c69923d87487bbf65f98ec323': ['0x10b16caad5e93b246b68453c7189674d19b172e6',
  '0x9b6b5d440b7d1606f8e058b063aaeede7b1e1d7a'],
 '06df701a49dc612adf7b9d377cfab3a1': ['0x2c251097063ca3f819364126d10bf9040c8d7468',
  '0x900873d1378c26a71bdd499553f470a97ff752a4',
  '0xaf29530805216fdb97009d389356ae362ba82d7e',
  '0xbd58cdf2266552d39ad8621793bd60c5b9f97e38',
  '0xe706b6071a5459fa34f7c85d938edd1912a4cf56'],
 '0853f718407b46d1fc77663afb5aca94': ['0x10834296fccc4ce31e86af11aae67873d56cdc54',
  '0xb062dc